In [1]:
import os
import sys

notebook_dir = os.path.abspath('')
sys.path.append(os.path.join(notebook_dir, '..'))

import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from keras_core import utils
from aepy.data.datasets import load_MNIST
from aepy.data.utils import evaluate
from aepy.models.base.default_architectures import (Decoder_Conv_MNIST, Decoder_MLP,
                                               Encoder_Conv_MNIST, Encoder_MLP)
from aepy.models.vae.vae_model import VAE
from aepy.pipelines.training import TrainingPipeline

Using TensorFlow backend


2023-11-23 14:27:13.534123: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load MNIST dataset
x_train, y_train, x_test, y_test = load_MNIST(persistant=True)

#x_train = np.expand_dims(x_train, -1).astype("float32") / 255
#x_test = np.expand_dims(x_test, -1).astype("float32") / 255

# Obtaint number of clasess
n_classes = len(set(y_train))

# Convert labels to categorical
y_train = utils.to_categorical(y_train, n_classes)
y_test = utils.to_categorical(y_test, n_classes)

2023-11-23 14:27:17 [INFO]: train-images-idx3-ubyte.gz already exists.
2023-11-23 14:27:17 [INFO]: train-labels-idx1-ubyte.gz already exists.
2023-11-23 14:27:17 [INFO]: t10k-images-idx3-ubyte.gz already exists.
2023-11-23 14:27:17 [INFO]: t10k-labels-idx1-ubyte.gz already exists.


In [3]:
train_data = dict(data=x_train.astype(float), labels=y_train)
test_data = dict(data=x_test.astype(float), labels=y_test)

In [4]:
# Model creation
model = VAE(input_dim=(x_train.shape[0], x_train.shape[1]), latent_dim=10, downstream_task='classification',
            encoder=Encoder_Conv_MNIST, decoder=Decoder_Conv_MNIST, layers_conf=[32, 64], n_classes=n_classes,
            weight_vae=0.25, weight_clf=20)

2023-11-23 14:27:17 [INFO]: Classificator available for the latent space of the autoencoder


In [5]:
pipe = TrainingPipeline(name='training_pipeline',
                        model=model, num_epochs=500, batch_size=128)

trained_model = pipe(train_data=train_data)

2023-11-23 14:27:17 [INFO]: +++ training_pipeline +++
2023-11-23 14:27:17 [INFO]: Creating folder in ../output_dir/training_pipeline_2023-11-23_14-27-17


Epoch 1/500


In [ ]:
def plot_label_clusters(vae, data, labels, include_flow=False):
    # Display a 2D plot of the digit classes in the latent space
    _, _, z = vae.encoder.predict(data)
    plt.figure(figsize=(12, 10))
    plt.scatter(z[:, 0], z[:, 1], c=labels)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()
    if include_flow:
      z = vae.flow.inverse(z)
      plt.figure(figsize=(12, 10))
      plt.scatter(z[:, 0], z[:, 1], c=labels)
      plt.colorbar()
      plt.xlabel("z[0]")
      plt.ylabel("z[1]")
      plt.show()

In [ ]:
plot_label_clusters(trained_model, x_train, y_train)
plot_label_clusters(trained_model, x_test, y_test)

In [ ]:
from sklearn.metrics import classification_report

y_train_predict = trained_model.predict(train_data)
y_test_predict =  trained_model.predict(test_data)

target_names = ['number 0', 'number 1', 'number 2', 'number 3', 'number 4', 'number 5', 'number 6', 'number 7', 'number 8', 'number 9']

classification_report(y_train, np.argmax(y_train_predict, axis=1), target_names=target_names)
classification_report(y_test, np.argmax(y_test_predict, axis=1), target_names=target_names)

In [ ]:
y_hat = trained_model.predict(test_data)

evaluate(y_true=np.argmax(test_data['labels'], axis=1), 
         y_hat=np.argmax(y_hat['clf'], axis=1),
         sel_metric=accuracy_score)